In [8]:
# created on April 21, 2022
# modified on April 22, 2022
# @author:          Ziyang Liu
# @email:           zliu96@uw.edu
# @organization:    Department of Urban Design and Planning, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S., and U.K. This script is modified from https://github.com/jakobzhao/geog458/tree/master/labs/lab02

In [ ]:
t="I am an idiot"
if "Ukraine" in t:
  print("True")

In [1]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [6]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets_uk.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = ""
    consumer_secret = ""
    access_token = ""
    access_token_secret = ""

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-6.6744, 50.0462, 1.4621, 59.1313]  # Contiguous US
                  # Hawaii

    stream_listener = StreamListener(time_limit=300, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1517199899028250625, 'username': 'revgpp', 'created_at': 'Thu Apr 21 17:53:38 +0000 2022', 'lng': -1.2074825, 'lat': 53.9318445, 'text': "A short stroll has turned into a much longer stroll &amp; I've discovered places that I've only ever driven past at spe… https://t.co/r9is6dk5PZ"}
{'id': 1517199900433395713, 'username': 'FPL_here_fishy', 'created_at': 'Thu Apr 21 17:53:39 +0000 2022', 'lng': 0.0640235, 'lat': 51.2010105, 'text': '@Linn_FPL Spanish national anthem.'}
{'id': 1517199904682266626, 'username': 'breggertelli', 'created_at': 'Thu Apr 21 17:53:40 +0000 2022', 'lng': -2.2038685, 'lat': 52.526056499999996, 'text': '@AddicksBlinders Zoom out there’ll be more empty’s'}
{'id': 1517199906175340545, 'username': 'chardonnay333', 'created_at': 'Thu Apr 21 17:53:40 +0000 2022', 'lng': -1.4796525, 'lat': 53.4651635, 'text': "@BBCNews I'm sure he will be fine this is 2022 not 1945 don't believe all you read in the news"}
{'id': 1517199908557791234, 'username': 'pup_ubu', 'create

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.


In [12]:
stream